<a href="https://colab.research.google.com/github/MicDso/-Machine-Learning-with-Python-Certification-Projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-27 12:43:24--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   159MB/s    in 0.2s    

2024-03-27 12:43:24 (159 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df = df_ratings.merge(df_books,on="isbn",how="left", sort = False)
df2 = (df.loc[(df["user"].isin(((df["user"].value_counts())[(df["user"].value_counts()) >= 200].index).values)) & (df["isbn"].isin(((df["isbn"].value_counts())[(df["isbn"].value_counts()) >= 100].index).values))]).drop_duplicates(['title', 'user'])
df_pivot = df2.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_csr = csr_matrix(df_pivot.values)
nb = NearestNeighbors(metric='cosine', leaf_size = 30, algorithm='auto').fit(df_csr)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    dist, idx = nb.kneighbors([df_pivot.iloc[df_pivot.index.get_loc(book)]], n_neighbors=6)
    recommended_books = [list((df_pivot.index[i], dist[0][j-4])) for i,j in zip(idx[0][::-1], range(4))]
    return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.7677075], ['The Weight of Water', 0.7699411], ['The Surgeon', 0.77085835], ['I Know This Much Is True', 0.8016211]]]
You passed the challenge! 🎉🎉🎉🎉🎉
